# Machine Learning with Distributed Computation

## 概述

**ML理论**
- 事实上，很多机器学习模型都可以写作形如下方的表达式：
$$ \min_\beta L(\beta )= \sum l_i (\beta) $$
- 其中$l_i$都是依赖于观测的，含有参数的函数
- 对于supervised learning，$l_i()$依赖于$(y_i, x_i)$且是光滑的（二阶可导）

**数据切分**
— 将数据按行切分：RDD分块（包含**部分观测**和**全部变量**）
- 相应的，上述的损失函数也有：$L(\beta) = \sum L_i(\beta) $

**计算思路**
1. 分布计算每个模块上的损失函数$L$以及梯度
2. 汇总分块结果，计算**总损失函数、梯度** (原理：梯度、损失函数都是线性可加的)
3. 利用优化算法更新参数

*上述思路可以解决GLM的全部问题*

**例外的情况**
- 目标函数不光滑
  - SVM
  - 包含绝对值的函数
    - LASSO回归
    - Least Absolute Deviations: $\min _x \left\| Y-X\beta \right\| _1$
    - 可以计算**次梯度**
- 参数存在约束
  - NNLS 非负最小二乘（约束OLS的参数$\beta \geq 0$)

***因此，分布式算法与优化算法是紧密相连的***

## ADMM 优化算法

- 概念：
  - 复杂优化算法（线性约束）
  - 通用形式：
  $$\min_{x,z} f(x)+g(z), ~s.t. Ax+Bz = c$$
  - 其中$x\in \R ^n , z \in \R ^m, f(),g()$为凸函数，$A[p\times n],B[p\times m]$，这里p可以理解为约束的个数
    - 凸函数
      - 二阶导数非负（Hessian矩阵非负定）
      - $f(t \mathbf{x}) \leq t(f(\mathbf{x})) , t \in [0 ,1 ]$ （函数图象要在函数上任意两点割线的下方（或重合））
      - 所有的凸函数的最小值是唯一的（最小值点或有多个），可通过优化算法收敛的
  - $Ax+Bz = c$等价于 $a_i'x+b_i'z=c_i ~, i = 1 , 2 \cdots p$，通常认为$p<m+n $

- 例：LAD 中位数回归（最小一乘）
  $$ \min _x \left\| Y-X\beta \right\| _1 $$
  - 稳健回归，对于异常值点的影响较小
  - 损失函数非光滑，但是凸函数
  - 可写作ADMM：
  $$ \text{set} f=0 , g= \left\| \cdot  \right\|_1 , z = Ax-b , B = -I, s.t. Ax-z = b $$

- 例：Lasso
  $$ \min_x = \frac{1}{2} \left\| Ax-b \right\| _2^2 + \lambda \left\| z \right\|_1 $$
  - ADMM：


- ADMM 算法流程
  $$\text{set } L_\rho(x,z,y) = f(x) + g(z) + y' (Ax+Bz-c)+(\rho / 2) \left\| Ax+Bz-c \right\|^2 $$

$$ x^{k+1} := \argmin_x L_\rho(x,z^k,y^k)  \\ z^{k+1} := \argmin_z L_\rho(x^{k+1},z,y^k)$$

      另外定义两类残差，当残差在设定误差内后收敛终止

- ADMM 的合理性讨论：
  - 算法的等价形式可见将原先的$x,z$共同优化函数分离为分别的两个优化问题（含有一个类似残差项的情况）
  - 在拆分之后有时该优化问题就会有显式解 
  - **因此问题便在于如何将优化问题转化为ADMM形式，并让分布更新有显式解**

- ADMM 适用于
  - 没有数值解
  - 目标函数一些地方不可求导
  - 含有线性约束
